In [26]:
import pandas as pd
import numpy as np
import random

import tensorflow as tf

from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.activations import relu, sigmoid
from keras.optimizers import Adam, SGD

from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

In [27]:
data = pd.read_csv('stand_scaled_data.csv')
col_list = data.columns.tolist()
features = col_list.copy()
features.remove('y')

In [60]:
def configure_nn(data, layers, 
                dropout_rate=0, 
                optimizer='Adam',
                loss='categorical_crossentropy',
                 kernel_initializer='lecun_normal',
                 kernel_regularizer=tf.keras.regularizers.L2(0.01)
                ):
    """ Layers argument shape:
    [[number of nodes, activate function], 
    [number of nodes, activate function],
    ...]
    """
    input_shape = len(data.columns.tolist()) - 1
    model = Sequential()
    
    model.add(Dense(layers[0][0], input_shape=(input_shape,),
                    activation=layers[0][1], kernel_initializer=kernel_initializer,
                   kernel_regularizer=kernel_regularizer))
    
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))
    
    for node in layers[1:]:
        model.add(Dense(node[0], activation=node[1], kernel_initializer=kernel_initializer, 
                        kernel_regularizer=kernel_regularizer))
        
    model.add(Dense(1, activation='sigmoid', kernel_initializer=kernel_initializer, 
                    kernel_regularizer=kernel_regularizer))
    
    model.compile(loss=loss, 
                 optimizer=optimizer,
                 metrics=tf.metrics.AUC(curve='ROC'))
    
    return model

In [63]:
def CVnet(df,
            layers,
            epochs=50, 
            n_splits=5,  
            rando_state=2021, 
            features=features, 
            if_print=True, 
            dropout_rate=0,
            optimizer='SGD',
            loss='binary_crossentropy', 
            batch_size=100,
            verbose=1,
            kernel_regularizer=tf.keras.regularizers.L2(0.01)):
    # Prepare KStratifiedKFOLD
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=rando_state)
    
    # Make copy of data
    data = df.copy()
    
    # Prepare empty lists
    train_results = []
    test_results = []
    preds = []
    
    # Prepare int to count fold s
    fold_number = 1
       
    for train, test in kf.split(data.index.values, data['y']):
        # Prepare model 
        model = configure_nn(data, layers, 
                dropout_rate=0, 
                optimizer='Adam',
                loss=loss,
                kernel_regularizer=kernel_regularizer)
        
        print(model.summary())
        
        model.fit(data.loc[train, features].values, data.loc[train, 'y'].values, 
                 epochs=epochs, 
                 validation_data=(data.loc[test, features].values, data.loc[test, 'y'].values),
                 batch_size=batch_size)
        
        # Make predictions
        train_preds = model.predict(data.loc[train, features])
        test_preds = model.predict(data.loc[test, features])
        preds.append(test_preds)
        
        indicies.append(df.iloc[test].index.tolist().copy())
        
        # Prepare ROC_AUC score
        train_roc = metrics.roc_auc_score(data.loc[train, 'y'], train_preds)
        test_roc = metrics.roc_auc_score(data.loc[test, 'y'], test_preds)
        
        # Add ROC_AUC to lis
        train_results.append(train_roc)
        test_results.append(test_roc)
        
        if if_print:
            print(f'FOLD NUMBER: {fold_number}')
            print(f'ROC_AUC ON TRAIN SCORE {train_roc}')
            print(f'ROC_AUC ON TEST SCORE {test_roc}')
                  
        fold_number += 1 
        
    return train_results, test_results, preds, ind

In [72]:
def nn_h_tuning():
    test_list = []
    for i in range(50):
        param_dict = {
            'optimizer' :random.choice(['Adam', 'SGD']),
            'batch_size': random.choice([50, 100, 200, 500]), 
            'dropout_rate': random.choice([0, 0.2, 0.4, 0.6]),
            'regulatization_rate': random.choice([0, 0.001, 0.05, 0.1]),
            'activation': random.choice(['relu', 'sigmoid', 'tanh']),
            'number_of_layers': random.randrange(1, 4)
        }
        
        layers = []
        for j in range(param_dict['number_of_layers']):
            
            layers.append([random.randrange(15, 100), param_dict['activation']])  
        train_results, test_results, preds  = CVnet(data, layers=layers,
                                                    optimizer=param_dict['optimizer'],
                                                    dropout_rate=param_dict['dropout_rate'],
                                                    batch_size=param_dict['batch_size'],
                                                    epochs=100,
                                                   kernel_regularizer=tf.keras.regularizers.L2(
                                                   param_dict['regulatization_rate']))
        
        test_list.append([param_dict.items(), layers, np.mean(test_results)])
        
    return test_list

In [ ]:
def test_features(data, features, **kwargs):
    

In [81]:
best_models = sorted(test_list, key=lambda x: x[-1], reverse= True)

In [86]:
best_models[5:]

[[dict_items([('optimizer', 'Adam'), ('batch_size', 100), ('dropout_rate', 0.4), ('regulatization_rate', 0.001), ('activation', 'relu'), ('number_of_layers', 1)]),
  [[85, 'relu']],
  0.9126243013629549],
 [dict_items([('optimizer', 'Adam'), ('batch_size', 200), ('dropout_rate', 0.4), ('regulatization_rate', 0.001), ('activation', 'tanh'), ('number_of_layers', 2)]),
  [[47, 'tanh'], [82, 'tanh']],
  0.9125956542539051],
 [dict_items([('optimizer', 'Adam'), ('batch_size', 500), ('dropout_rate', 0.6), ('regulatization_rate', 0.001), ('activation', 'tanh'), ('number_of_layers', 1)]),
  [[47, 'tanh']],
  0.9118117298511524],
 [dict_items([('optimizer', 'SGD'), ('batch_size', 500), ('dropout_rate', 0.2), ('regulatization_rate', 0), ('activation', 'tanh'), ('number_of_layers', 2)]),
  [[65, 'tanh'], [84, 'tanh']],
  0.9117295406108165],
 [dict_items([('optimizer', 'SGD'), ('batch_size', 500), ('dropout_rate', 0.6), ('regulatization_rate', 0.001), ('activation', 'tanh'), ('number_of_layers', 

In [ ]:
test_list = nn_h_tuning()

Model: "sequential_89"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_300 (Dense)            (None, 17)                1717      
_________________________________________________________________
dense_301 (Dense)            (None, 89)                1602      
_________________________________________________________________
dense_302 (Dense)            (None, 35)                3150      
_________________________________________________________________
dense_303 (Dense)            (None, 1)                 36        
Total params: 6,505
Trainable params: 6,505
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
391/391 [==============================] - 1s 2ms/step - loss: 0.5293 - auc_89: 0.6868 - val_loss: 0.3423 - val_auc_89: 0.8979
Epoch 2/100
391/391 [==============================] - 0s 1ms/step - loss: 0.3328 - auc_89: 0.9018 - val_

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



79/79 [==============================] - 0s 2ms/step - loss: 0.3494 - auc_212: 0.9060 - val_loss: 0.3530 - val_auc_212: 0.9013
Epoch 64/100
79/79 [==============================] - 0s 2ms/step - loss: 0.3453 - auc_212: 0.9079 - val_loss: 0.3531 - val_auc_212: 0.9013
Epoch 65/100
79/79 [==============================] - 0s 2ms/step - loss: 0.3461 - auc_212: 0.9075 - val_loss: 0.3533 - val_auc_212: 0.9014
Epoch 66/100
79/79 [==============================] - 0s 2ms/step - loss: 0.3431 - auc_212: 0.9081 - val_loss: 0.3531 - val_auc_212: 0.9014
Epoch 67/100
79/79 [==============================] - 0s 2ms/step - loss: 0.3482 - auc_212: 0.9053 - val_loss: 0.3533 - val_auc_212: 0.9014
Epoch 68/100
79/79 [==============================] - 0s 2ms/step - loss: 0.3436 - auc_212: 0.9062 - val_loss: 0.3535 - val_auc_212: 0.9015
Epoch 69/100
79/79 [==============================] - 0s 2ms/step - loss: 0.3477 - auc_212: 0.9072 - val_loss: 0.3535 - val_auc_212: 0.9015
Epoch 70/100
79/79 [=============

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



782/782 [==============================] - 1s 1ms/step - loss: 0.5415 - auc_227: 0.8829 - val_loss: 0.5418 - val_auc_227: 0.8793
Epoch 31/100
782/782 [==============================] - 1s 1ms/step - loss: 0.5377 - auc_227: 0.8848 - val_loss: 0.5406 - val_auc_227: 0.8824
Epoch 32/100
782/782 [==============================] - 1s 1ms/step - loss: 0.5387 - auc_227: 0.8819 - val_loss: 0.5404 - val_auc_227: 0.8784
Epoch 33/100
782/782 [==============================] - 1s 1ms/step - loss: 0.5416 - auc_227: 0.8833 - val_loss: 0.5422 - val_auc_227: 0.8812
Epoch 34/100
782/782 [==============================] - 1s 1ms/step - loss: 0.5374 - auc_227: 0.8814 - val_loss: 0.5410 - val_auc_227: 0.8782
Epoch 35/100
782/782 [==============================] - 1s 1ms/step - loss: 0.5398 - auc_227: 0.8829 - val_loss: 0.5404 - val_auc_227: 0.8812
Epoch 36/100
782/782 [==============================] - 1s 996us/step - loss: 0.5399 - auc_227: 0.8827 - val_loss: 0.5419 - val_auc_227: 0.8804
Epoch 37/100
782/